In [ ]:
#8.1 改变对象的字符串显示

class Pair:
    def __init__(self,x,y):
        self.x=x
        self.y=y
    def __repr__(self):#                show Pair() will use this func!!!
        return 'Pair({}, {})'.format(self.x,self.y)
#         return 'Pair({0.x!r}, {0.y!r})'.format(self) same!!!!!!!!!!!!!!!
    def __str__(self):#                 print() and str() will use this func!!!
        return '({}, {})'.format(self.x,self.y)
"""
>>> p = Pair(3, 4)
>>> p
Pair(3, 4) # __repr__() output
>>> print(p)
(3, 4) # __str__() output
"""
p=Pair(3,4)
print('p is {0!r}'.format(p))#use __repr__
print('p is {0}'.format(p))#  use __str__


In [ ]:
# 8.2 自定义字符串的格式化

#格式化是定义在类之上的
_formats={
    'ymd': '{d.year}-{d.month}-{d.day}',
    'dmy':'{d.day}/{d.month}/{d.year}'
}
class Date:
    def __init__(self, year, month,day):
        self.year=year
        self.month=month
        self.day=day
    
    def __format__(self, code):
        if code == '':
            code = 'dmy'
        fmt = _formats[code]
        return fmt.format(d=self)
d=Date(2018, 12,21)
print(format(d,'ymd'))
print("The date is {:dmy}".format(d))

In [ ]:
# 8.3 上下文管理协议 就是with...open...as...

from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address=address
        self.family=family
        self.type=type
        self.sock=None
        self.connections=[]#This is a stack used to control multi connections
    
    def __enter__(self):
        if self.sock is not None:
            raise RuntimeError('Already connected!')
        self.sock = socket(self.family, self.type)
        self.sock.connect(self.address)
        self.connections.append(sock)
        return self.sock
    
    def __exit__(self, exc_ty,exc_val,tb):
#         self.sock.close()
        self.connections.pop().close()
        self.sock=None

        
from functools import partial

conn = LazyConnection(('www.python.org',80))
with conn as s:
    # conn.__enter__() executes: connection open
    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host: www.python.org\r\n')
    s.send(b'\r\n')
    resp = b''.join(iter(partial(s.recv, 8192), b''))
    # conn.__exit__() executes: connection closed

In [ ]:
#8.4 创建大量对象时节省内存方法

class Date:
    __slot__=['year','month','day']
    def __init__(self, year, month, day):
        self.year = year
        self.month= month
        self.day=day
 #       关于 __slots__ 的一个常见误区是它可以作为一个封装工具来防止用户给实例增加新的属性。
# 尽管使用slots可以达到这样的目的，但是这个并不是它的初衷。 __slots__ 更多的是用来作为一个内存优化工具。

In [ ]:
#8.5 在类中封装属性名
class A:
    def __init__(self):
        self._internal=0
        self.public=1
    def public_method(self):
        pass
    def _internal_method(self):
        pass
"""
_单下划线表示私有属性或私有方法
__双下划线表示私有属性或私有方法而且不希望子类继承,sys会自动改名字_A__private
"""
lambda_ = 2.0 # Trailing _ to avoid clash with lambda keyword

In [ ]:
#8.6 创建可管理的属性

class Person:
    def __init__(self, first_name):
        self.first_name = first_name#写的没错,这是先访问setter function!!!! line 32!!!!
    
    #getter function
    @property
    def first_name(self):
        return self._first_name
    
    #setter function
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string!')
        self._first_name=value
    
    #Delete function
    @first_name.deleter
    def first_name(self):
        raise AttributeError('Cant delete attribute!')
a=Person(111)
# a.first_name='zhazha'
print(a.first_name)
# del a.first_name
print('============')
a._first_name

class Person:
    def __init__(self, first_name):
        self.set_first_name(first_name)

    # Getter function
    def get_first_name(self):
        return self._first_name

    # Setter function
    def set_first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value

    # Deleter function (optional)
    def del_first_name(self):
        raise AttributeError("Can't delete attribute")

    # Make a property from existing get/set methods
name = property(get_first_name, set_first_name, del_first_name)

"""
这是用途之一!!!!!!!!!!
"""
import math
class Circle:
    def __init__(self, radius):
        self.radius = radius

    @property
    def area(self):
        return math.pi * self.radius ** 2

    @property
    def diameter(self):
        return self.radius * 2

    @property
    def perimeter(self):
        return 2 * math.pi * self.radius
"""
>>> c = Circle(4.0)
>>> c.radius
4.0
>>> c.area  # Notice lack of ()
50.26548245743669
>>> c.perimeter  # Notice lack of ()
25.132741228718345
>>>
"""

In [ ]:
#8.7 调用父类方法
class A:
    def spam(self):
        print('A.spam')

class B(A):
    def spam(self):
        print('B.spam')
        super().spam()  # Call parent spam()
"""
确保父类被正确初始化
"""
class A:
    def __init__(self):
        self.x = 0

class B(A):
    def __init__(self):
        super().__init__()
        self.y = 1
b=B()
print(b.x)
print('========================================================')
"""
子类会先于父类被检查
多个父类会根据它们在列表中的顺序被检查
如果对下一个类存在两个合法的选择，选择第一个父类
"""
class Base:
    def __init__(self):
        print('Base.__init__')

class A(Base):
    def __init__(self):
        super().__init__()
        print('A.__init__')

class B(Base):
    def __init__(self):
        super().__init__()
        print('B.__init__')

class C(A,B):
    def __init__(self):
        super().__init__()  # Only one call to super() here
        print('C.__init__')
print(C())


In [ ]:
#8.8 子类中扩展property
class Person:
    def __init__(self, name):
        self.name = name

    # Getter function
    @property
    def name(self):
        return self._name

    # Setter function
    @name.setter
    def name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._name = value

    # Deleter function
    @name.deleter
    def name(self):
        raise AttributeError("Can't delete attribute")
#扩展属性
class SubPerson(Person):
    @Person.name.getter
    def name(self):
        print('Getting name')
        return super().name
    
    @Person.name.setter
    def name(self, value):
        print('Setting name to', value)
        super(SubPerson, SubPerson).name.__set__(self, value)

In [ ]:
#8.9 创建新的类或实例属性------描述器

class Integer:
    def __init__(self, name):
        self.name = name

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]

    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise TypeError('Expected an int')
        instance.__dict__[self.name] = value

    def __delete__(self, instance):
        del instance.__dict__[self.name]
        
class Point:
    x=Integer('x')
    y=Integer('y')
    
    def __init__(self,x,y):
        self.x=x
        self.y=y
p=Point(2,3)
"""
>>> p = Point(2, 3)
>>> p.x # Calls Point.x.__get__(p,Point)
2
>>> p.y = 5 # Calls Point.y.__set__(p, 5)
>>> p.x = 2.3 # Calls Point.x.__set__(p, 2.3)
Traceback (most recent call last):
    File "<stdin>", line 1, in <module>
    File "descrip.py", line 12, in __set__
        raise TypeError('Expected an int')
TypeError: Expected an int
>>>
"""
"""
描述器的一个比较困惑的地方是它只能在类级别被定义，而不能为每个实例单独定义。因此，下面的代码是无法工作的：
"""
"""
最后不是很理解
"""

In [ ]:
#8.10  使用延迟计算属性------访问后计算并保留结果

class lazyproperty:#this is a discriptor!
    def __init__(self,func):
        self.func=func
    
    def __get__(self,instance,cls):
        if instance is None:
            return self
        else:
            value = self.func(instance)
            setattr(instance, self.func.__name__, value)
            return value

import math

class Circle:
    def __init__(self, radius):
        self.radius = radius

    @lazyproperty#how to use the discriptor!
    def area(self):
        print('Computing area')
        return math.pi * self.radius ** 2

    @lazyproperty
    def perimeter(self):
        print('Computing perimeter')
        return 2 * math.pi * self.radius
"""
>>> c = Circle(4.0)
>>> c.radius
4.0
>>> c.area
Computing area
50.26548245743669
>>> c.area
50.26548245743669
>>> c.perimeter
Computing perimeter
25.132741228718345
>>> c.perimeter
25.132741228718345
>>>
"""
def lazyproperty(func):#不会被改属性大小!!!!!!比上面的要好,但效率要低一些!
    name = '_lazy_' + func.__name__
    @property
    def lazy(self):
        if hasattr(self, name):
            return getattr(self, name)
        else:
            value = func(self)
            setattr(self, name, value)
            return value
    return lazy

In [1]:
#8.11  简化数据结构的初始化
import math

class Structure1:#                               先写一个公用的__init__函数
    # Class variable that specifies expected fields
    _fields = []

    def __init__(self, *args):
        if len(args) != len(self._fields):
            raise TypeError('Expected {} arguments'.format(len(self._fields)))
        # Set the arguments
        for name, value in zip(self._fields, args):
            setattr(self, name, value)

# Example class definitions
class Stock(Structure1):
    _fields = ['name', 'shares', 'price']

class Point(Structure1):
    _fields = ['x', 'y']

class Circle(Structure1):
    _fields = ['radius']

    def area(self):
        return math.pi * self.radius ** 2
"""
然后让子类继承就好了~
"""

'\n然后让子类继承就好了~\n'

In [2]:
#8.12 定义接口或者抽象基类

pass 
#interface 感觉不是很实用的yangzi~

In [3]:
#8.13 实现数据模型的类型约束

#完``````````````````````